**This notebook contains a scripts that creates a database of download links for student documents**

**Import Libraries**

In [172]:
import pandas as pd
import numpy as np
import gspread
import gspread_dataframe as gd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from fuzzywuzzy import process

**Connect to Google Drive**

In [173]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Authenticate Gspread**

In [193]:
gc = gspread.oauth()

**Create WorkSheets**

In [194]:
#2020 student files database worksheet
cohort_2020_files_worksheet = gc.open("Student Files Database").worksheet("2020")

**List Files in Folder**

In [176]:
#create list of files or folders in drive folder
def list_files(file_id):
    file_list = drive.ListFile(
        {
            'q': "'{}' in parents and trashed=false".format(file_id),
            'corpora': "teamDrive",
            'teamDriveId': "0ALGjY-PCeStEUk9PVA",
            'includeTeamDriveItems': "true",
            'supportsTeamDrives': "true"
        }
    ).GetList()
    return file_list

#create list to store folder ids
campus_folders = []
student_folders = []
student_files = {}

#create initial file list
file_list = list_files('13ovdZDNKpNfE4dj38hXH2qFwtbz_BWe3')

#save folder id's in campus list
for campus in file_list:
    campus_folders.append(campus['id'])


In [177]:
#save folder id's in student folders list   
for folder_id in campus_folders:
    file_list = list_files(folder_id)
    for student in file_list:
        if 'dismissed' not in student['title'].lower():
            student_folders.append(student['id'])


In [178]:
#save file id and title in student file dict 
for folder_id in student_folders:
    file_list = list_files(folder_id)
    file_params = {}
    for file in file_list:
        parent_id = file['parents'][0]['id']
        file_id = file['id']
        file_params = {"file_id": file_id, "parent_id": parent_id}
        student_files[file['title']] = file_params


**Convert Student Files Dict to Dataframe**

In [179]:
student_files_df = pd.DataFrame(list(student_files.items()),columns = ['Filename','File_id'])
master_files_df =  pd.DataFrame(list(student_files.items()),columns = ['Filename','File_id'])

**Convert Link IDs to Links**

In [180]:
#student_files_df['File_id'] = student_files_df['File_id'].apply(lambda x: f"https://drive.google.com/uc?export=download&id={x}")
#master_files_df['File_id'] = master_files_df['File_id'].apply(lambda x: f"https://drive.google.com/uc?export=download&id={x}")
#student_files_df = master_files_df.copy()
#print(student_files)

**Subset Dataframe to Contain Specific Documents Only**

In [181]:
#method to find specific docs
def find_files(full_file_name):
    file_name = full_file_name.split(".")[0]
    file_purpose = file_name.split("_")[-1]
    file_purpose = file_purpose.strip()
    file_purpose = file_purpose.lower()
    
    if file_purpose == 'snr cert':
        return True
    if file_purpose == 'id':
        return True
    if file_purpose == 'wtc contract':
        return True
    return False

#apply find files method to filenames column
student_files_df = student_files_df[student_files_df['Filename'].apply(find_files)]
student_files_df

,Filename,File_id
0,Innocent Sandla_Snr Cert.pdf,{'file_id': '1HG6_4HaXd2lDEBYF8Z-n3jKstX6dvSJJ...
1,Innocent Sandla_WTC Contract.pdf,{'file_id': '155f3dqgYEdvf9HZvh-tQhvyDnkIBkUYF...
2,Innocent Sandla_ID.pdf,{'file_id': '1o57FHWu1uPvs8UWz1bvC1JSZVeqljL8_...
6,Sebaratlane Mashaba_WTC Contract.pdf,{'file_id': '1kwHFmZ8duYfKscdcT7Y1-dgZscevG7rZ...
7,Sebaratlane Mashaba_Snr Cert.pdf,{'file_id': '1yXH2UEEDdqF7MjlDvK3GkPSdZJgZZtTR...
...,...,...
1849,Andrew Cordoms_ID.pdf,{'file_id': '1mygBIRivKYA152p4C8LXGs3mmBhbawT5...
1852,Andrew Cordoms_WTC Contract.pdf,{'file_id': '10A2Azs81VmHT33PP5eq17IRC-E3uw77c...
1853,Aidan Josias_Snr Cert.PDF,{'file_id': '1uHpxvywgwWGmijdB-wQZ6wuFbyLBE0HY...
1854,Aidan Josias_WTC Contract.pdf,{'file_id': '1VZb34hnqSs_aOTAYayTvob8d3mVOy6YE...


**Split Filename column into Fullname, G12_cert and Contract Columns**

In [182]:
#strip filename whitespaces
student_files_df['Filename'] = student_files_df['Filename'].str.strip()

#set filenames to lowercase
student_files_df['Filename'] = student_files_df['Filename'].str.lower()

#remove file extension
student_files_df['Filename'] = [elem[0] for elem in student_files_df['Filename'].str.split(".")]

#split filename into fullname
student_files_df['Fullname'] = [elem[0] for elem in student_files_df['Filename'].str.split("_")]

#split filename into filetype
student_files_df['Filetype'] = [elem[-1] for elem in student_files_df['Filename'].str.split("_")]

#strip fullname and filetype whitespaces
student_files_df['Fullname'] = student_files_df['Fullname'].str.strip()
student_files_df['Filetype'] = student_files_df['Filetype'].str.strip()

#replace filetype values
student_files_df['Filetype'] = student_files_df['Filetype'].replace({'id': 'ID file', 'snr cert': 'G12 Certification',
                             'wtc contract': 'WTC Contract'})


**Reshape Data Frame**

In [183]:
#drop Filename column
student_files_df = student_files_df[['Fullname', 'Filetype', 'File_id']]

#use pivot to reshape dataframe
student_files_df = student_files_df.pivot(index='Fullname',columns='Filetype',values='File_id')

#remove column 'filetype'
student_files_df = student_files_df.rename_axis(columns = None)

#remove index
student_files_df = student_files_df.reset_index()


In [184]:
student_files_df['WTC Contract'].values

array([{'file_id': '1C7g17p3ZGdifgxYA1Ej8uv0HOrcJCyN7', 'parent_id': '13QrdFDlWkhdTo1Eus031e3hf5dtUlXRW'},
       {'file_id': '1v2FE2qdaTl3yHqnc5cOxNME4GFEWoskI', 'parent_id': '17ZoNVnnE1nXx0RBWa_z-LH2acgnVR0w8'},
       {'file_id': '1tLuQ4IYLvZN_l75U_Ew7xy8kPX5if3SQ', 'parent_id': '1IY2VD1Tjrp-sSAh2ydkTAEKCm1OMAtY8'},
       {'file_id': '1S4nEgPOn26HEFvijtBaFzGnspqAyzBf3', 'parent_id': '1cvTA0KbK27LJixsLq7Wcm9ZtSF59Zzbx'},
       nan,
       {'file_id': '1Odj94GM31Va8bAXVqfIB49yLRkQUTVh_', 'parent_id': '18gA3lQlfHiSi_8Fqf17BGgnOSs6Szvv1'},
       {'file_id': '1VZb34hnqSs_aOTAYayTvob8d3mVOy6YE', 'parent_id': '1GUU-6-DLCl81L14WjKJ5lEXhdhwzYKD7'},
       {'file_id': '1M9qEhRO39gdyhmRyCBPWHcQ7HZPb_lCA', 'parent_id': '1AQ_8ymUMoJ3xdVoA4GEptXOfmrZTt7jH'},
       {'file_id': '1UKoMGrZpe53urAR4fv8o5Me4KQPyVpvx', 'parent_id': '1SkC0uEx4ajIB54JOpo18v2iYMcDlGJav'},
       nan,
       {'file_id': '164Z8Z0ZyoTt1wSffZ3gUN-XlY3n8Tari', 'parent_id': '1RXpqlVKVIno82xqehud0G0h939FCevww'},
       {'file

**Search for Missing links**

In [185]:
'''#function to search master sheet for missing links
def search_link(name, file_type):
    last_name = name.split(" ")[-1]
    last_name = last_name.strip().lower()
    file_type = file_type.strip().lower()
    for file in master_files_df["Filename"]:
        file = file.strip().lower()
        if last_name in file:
            if file_type in file:
                id_list = master_files_df.loc[master_files_df["Filename"].str.strip().str.lower() == file, "File_id"].values
                if len(id_list) == 1:
                    return id_list[0]
                else:
                    return None

#search for missing contract links
results_list = [search_link(name, 'contract') for name in student_files_df.loc[student_files_df['WTC Contract'].isna(), 'Fullname']]
student_files_df.loc[student_files_df['WTC Contract'].isna(), 'WTC Contract'] = results_list

#search for missing grade 12 certificate links
results_list = [search_link(name, 'cert') for name in student_files_df.loc[student_files_df['G12 Certification'].isna(), 'Fullname']]
student_files_df.loc[student_files_df['G12 Certification'].isna(), 'G12 Certification'] = results_list

#search for missing id links
results_list = [search_link(name, '_id') for name in student_files_df.loc[student_files_df['ID file'].isna(), 'Fullname']]
student_files_df.loc[student_files_df['ID file'].isna(), 'ID file'] = results_list'''

'#function to search master sheet for missing links\ndef search_link(name, file_type):\n    last_name = name.split(" ")[-1]\n    last_name = last_name.strip().lower()\n    file_type = file_type.strip().lower()\n    for file in master_files_df["Filename"]:\n        file = file.strip().lower()\n        if last_name in file:\n            if file_type in file:\n                id_list = master_files_df.loc[master_files_df["Filename"].str.strip().str.lower() == file, "File_id"].values\n                if len(id_list) == 1:\n                    return id_list[0]\n                else:\n                    return None\n\n#search for missing contract links\nresults_list = [search_link(name, \'contract\') for name in student_files_df.loc[student_files_df[\'WTC Contract\'].isna(), \'Fullname\']]\nstudent_files_df.loc[student_files_df[\'WTC Contract\'].isna(), \'WTC Contract\'] = results_list\n\n#search for missing grade 12 certificate links\nresults_list = [search_link(name, \'cert\') for name i

In [186]:
'''#function to search master sheet for missing links using variations
def search_link_2(name, file_type):
    last_name = name.split(" ")[-1]
    last_name = last_name.strip()
    file_type = file_type.strip()
    for file in master_files_df["Filename"]:
        file = file.strip()
        if last_name in file.lower():
            if file_type in file:
                id_list = master_files_df.loc[master_files_df["Filename"].str.strip().str.lower() == file.strip().lower(), "File_id"].values
                if len(id_list) == 1:
                    return id_list[0]
                else:
                    return None
                
#search for missing 'WTC' contract links
results_list = [search_link_2(name, 'WTC') for name in student_files_df.loc[student_files_df['WTC Contract'].isna(), 'Fullname']]
student_files_df.loc[student_files_df['WTC Contract'].isna(), 'WTC Contract'] = results_list

#search for missing grade 12 certificate 'Snr' links
results_list = [search_link_2(name, 'Snr') for name in student_files_df.loc[student_files_df['G12 Certification'].isna(), 'Fullname']]
student_files_df.loc[student_files_df['G12 Certification'].isna(), 'G12 Certification'] = results_list

#search for missing 'ID' links
results_list = [search_link_2(name, 'ID') for name in student_files_df.loc[student_files_df['ID file'].isna(), 'Fullname']]
student_files_df.loc[student_files_df['ID file'].isna(), 'ID file'] = results_list

#search for missing 'I.D' links
results_list = [search_link_2(name, 'ID') for name in student_files_df.loc[student_files_df['ID file'].isna(), 'Fullname']]
student_files_df.loc[student_files_df['ID file'].isna(), 'ID file'] = results_list'''

'#function to search master sheet for missing links using variations\ndef search_link_2(name, file_type):\n    last_name = name.split(" ")[-1]\n    last_name = last_name.strip()\n    file_type = file_type.strip()\n    for file in master_files_df["Filename"]:\n        file = file.strip()\n        if last_name in file.lower():\n            if file_type in file:\n                id_list = master_files_df.loc[master_files_df["Filename"].str.strip().str.lower() == file.strip().lower(), "File_id"].values\n                if len(id_list) == 1:\n                    return id_list[0]\n                else:\n                    return None\n                \n#search for missing \'WTC\' contract links\nresults_list = [search_link_2(name, \'WTC\') for name in student_files_df.loc[student_files_df[\'WTC Contract\'].isna(), \'Fullname\']]\nstudent_files_df.loc[student_files_df[\'WTC Contract\'].isna(), \'WTC Contract\'] = results_list\n\n#search for missing grade 12 certificate \'Snr\' links\nresult

**Search for Misspelt Names**

In [187]:
'''#function to search master sheet for misspelt names
def search_misspelt_names(areas, strOptions, student_files_df):
    for area in areas:
        highest = process.extractOne(area, strOptions)
        if highest[1] > 70:
            condition_1 = student_files_df['Fullname'] == highest[0]
            condition_2 = student_files_df['Fullname'] == area
            index_id = student_files_df[condition_1].index.values[0]
            contract_link = student_files_df.loc[condition_1,'WTC Contract'].values[0]
            cert_link = student_files_df.loc[condition_1,'G12 Certification'].values[0]
            id_link = student_files_df.loc[condition_1,'ID file'].values[0]
            if contract_link is not None:
                student_files_df.loc[condition_2, 'WTC Contract'] = contract_link
            if cert_link is not None:
                student_files_df.loc[condition_2, 'G12 Certification'] = cert_link
            if id_link is not None:
                student_files_df.loc[condition_2, 'ID file'] = id_link
            student_files_df = student_files_df.drop(index_id)
    return student_files_df

#search for misspelt names in the contract column
areas = [name for name in student_files_df.loc[student_files_df['WTC Contract'].isna(), 'Fullname']]
strOptions = [name for name in student_files_df.loc[~student_files_df['WTC Contract'].isna(), 'Fullname']]
student_files_df = search_misspelt_names(areas, strOptions, student_files_df)

#search for misspelt names in the id column
areas = [name for name in student_files_df.loc[student_files_df['ID file'].isna(), 'Fullname']]
strOptions = [name for name in student_files_df.loc[~student_files_df['ID file'].isna(), 'Fullname']]
student_files_df = search_misspelt_names(areas, strOptions, student_files_df)

#search for misspelt names in the g12 certificate column
areas = [name for name in student_files_df.loc[student_files_df['G12 Certification'].isna(), 'Fullname']]
strOptions = [name for name in student_files_df.loc[~student_files_df['G12 Certification'].isna(), 'Fullname']]
student_files_df = search_misspelt_names(areas, strOptions, student_files_df)'''

"#function to search master sheet for misspelt names\ndef search_misspelt_names(areas, strOptions, student_files_df):\n    for area in areas:\n        highest = process.extractOne(area, strOptions)\n        if highest[1] > 70:\n            condition_1 = student_files_df['Fullname'] == highest[0]\n            condition_2 = student_files_df['Fullname'] == area\n            index_id = student_files_df[condition_1].index.values[0]\n            contract_link = student_files_df.loc[condition_1,'WTC Contract'].values[0]\n            cert_link = student_files_df.loc[condition_1,'G12 Certification'].values[0]\n            id_link = student_files_df.loc[condition_1,'ID file'].values[0]\n            if contract_link is not None:\n                student_files_df.loc[condition_2, 'WTC Contract'] = contract_link\n            if cert_link is not None:\n                student_files_df.loc[condition_2, 'G12 Certification'] = cert_link\n            if id_link is not None:\n                student_file

**Convert NAN values to NONE**

In [188]:
#convert g12 certs nan values to none
student_files_df['G12 Certification'] = student_files_df['G12 Certification'].replace({np.nan: None})

#convert id file nan values to none
student_files_df['ID file'] = student_files_df['ID file'].replace({np.nan: None})

#convert wtc contract nan values to none
student_files_df['WTC Contract'] = student_files_df['WTC Contract'].replace({np.nan: None})

**Create Column Containing Parent Folder ID**

In [189]:
for name in student_files_df['Fullname']:
    condition = student_files_df['Fullname'] == name
    temp_series = student_files_df.loc[condition,:]
    if temp_series['G12 Certification'].values[0] is not None:
        student_files_df.loc[condition, 'Parent ID'] = temp_series['G12 Certification'].values[0]['parent_id']
        continue
    if temp_series['ID file'].values[0] is not None:
        student_files_df.loc[condition, 'Parent ID'] = temp_series['ID file'].values[0]['parent_id']
        continue
    if temp_series['WTC Contract'].values[0] is not None:
        student_files_df.loc[condition, 'Parent ID'] = temp_series['WTC Contract'].values[0]['parent_id']
        continue

student_files_df.head()

,Fullname,G12 Certification,ID file,WTC Contract,Parent ID
0,abdullah ebrahim,None,{'file_id': '1slAYT2SdS0AOLAYse8RLPu6vaMCgO7ui...,{'file_id': '1C7g17p3ZGdifgxYA1Ej8uv0HOrcJCyN7...,13QrdFDlWkhdTo1Eus031e3hf5dtUlXRW
1,abdur-raheem lee,{'file_id': '1x8pKVL8q4UtkVOI6EB3mNTQW1ftVh0NU...,{'file_id': '1tNyYGleV59vFuzwdjsQF4bIp5QEVdB3c...,{'file_id': '1v2FE2qdaTl3yHqnc5cOxNME4GFEWoskI...,17ZoNVnnE1nXx0RBWa_z-LH2acgnVR0w8
2,abigail hlalele,{'file_id': '1drznpNwHbj4q9P3BpONLG7SX-CpohVKT...,{'file_id': '1Kb_29so7-BqhK2K4A7nt83te3bXaoscM...,{'file_id': '1tLuQ4IYLvZN_l75U_Ew7xy8kPX5if3SQ...,1IY2VD1Tjrp-sSAh2ydkTAEKCm1OMAtY8
3,adam becker,{'file_id': '1d5haUDWJFo6gEPUjHMR9djJOJG-w_sFr...,{'file_id': '17G0RyMw5xxul4bofHMArFVdF5l1uFEo_...,{'file_id': '1S4nEgPOn26HEFvijtBaFzGnspqAyzBf3...,1cvTA0KbK27LJixsLq7Wcm9ZtSF59Zzbx
4,adams wren,{'file_id': '1hBNdjBTjTcI01z3twXcJNhVqJncPJjlW...,{'file_id': '1mljxufELpRpqLc26NNiPzBJyw20rwZDE...,None,1Bjh4L5VB8sPTwnPNmMafUxAfm64r-tg0


**Function to Unpack File IDs**

In [190]:
def unpack_ids(file_params):
    if file_params is not None:
        return file_params['file_id']
    return None

In [191]:
#unpack coontract
student_files_df['WTC Contract'] = student_files_df['WTC Contract'].apply(unpack_ids)

#unpack id
student_files_df['ID file'] = student_files_df['ID file'].apply(unpack_ids)

#unpack certification
student_files_df['G12 Certification'] = student_files_df['G12 Certification'].apply(unpack_ids)

**Upload Dataframe to Student Files Database**

In [195]:
#export cohort file ids
gd.set_with_dataframe(cohort_2020_files_worksheet, student_files_df)